## 二次元シュレーディンガー方程式の解：直交座標系で解く
今回は、二次元シュレーディンガー方程式をもう少し力技で解いてみよう。
二次元シュレーディンガー方程式は
$$
\left( -\frac{\hbar^2}{2m} {\bf \nabla}^2 + V({\bf r}) \right)\psi({\bf r}) = \epsilon \psi({\bf r})
$$
となる。ここで、ナブラ演算子${\bf \nabla}^2$は、直交座標系では
$$
{\bf \nabla}^2 = \frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2}
$$
である。

これまでは系に回転対称性があるとして円筒座標系を用いて解いた。ここでは、回転対称性を課さずに、直交座標系で直接解いてみよう。
これまでのノートと同じようにするために、半径$R=\sqrt{x^2+y^2}$の円を考え、それより外側には非常に大きなポテンシャルがあるとする。それ以外にはポテンシャルはないとする。つまり、これまでのノートでのポテンシャルがない場合に相当する。
計算する領域は幅$L$の正方形とする。
このとき、
二階微分をQM01と同様に
$$
\frac{d^2}{dx^2}\psi |_{x=x_i} \sim \frac{\psi(x_i + a) -2\psi(x_i) +\psi(x_i - a)}{a^2}
$$
と差分化する。yも同様である。半径$R$よりも遠い領域ではポテンシャルを強くいれるため、正方形の境界条件はどのようにしてもよい。ここでは、周期境界条件としておく。

行列は

In [1]:
using LinearAlgebra
using SparseArrays 
function make_matH(Nx,Ny,V,L)
        N = Nx*Ny
        mat_H = spzeros(ComplexF64,N,N)
        ax = L/(Nx-1)
        ay = L/(Ny-1)
        for ix=1:Nx
            x = ax*(ix-1)-L/2
            for iy=1:Ny
                i = (iy-1)*Nx+ix
                y = ay*(iy-1)-L/2
                for dx = -1:1
                    jx = ix + dx
                    jx +=ifelse(jx > Nx,-Nx,0)
                    jx +=ifelse(jx < 1,Nx,0)
                    tx = 0.0
                    ty = 0.0
                
                    if dx==0
                        tx = -2.0
                        for dy = -1:1
                            jy = iy + dy
                            jy +=ifelse(jy > Ny,-Ny,0)
                            jy +=ifelse(jy < 1,Ny,0)
                            j = (jy-1)*Nx + jx
                            if dy == 0
                                ty = -2.0
                                mat_H[i,j] = -(tx/ax^2+ty/ay^2)+V(x,y)
                            else
                                ty = 1.0
                                mat_H[i,j] = -(ty/ay^2)
                            end
                        end
                    else
                        jy = iy
                        j = (jy-1)*Nx + jx
                        tx = 1.0
                        mat_H[i,j] = -(tx/ax^2)
                    end
                                        
                    
                end
            end
        end
        return mat_H
    end

make_matH (generic function with 1 method)

ここで、二次元座標系を考えるために行列のサイズは$N_x \times N_y$と大きなものになるため、疎行列を使うこととした。
疎行列とは、行列のうちゼロではない要素のみを格納した行列のことであり、メモリと計算量を節約することができる。
疎行列の対角化にはArpackを用いるので、

In [2]:
using Arpack

としておく。

ポテンシャルは、半径R0以上で非常に大きな値をとる関数：

In [3]:
const R0 = 10.0
const μ = 0.0
function V(x,y)
    r2=x^2+y^2
    v = -μ
    if r2 > R0^2
        v += 10000.0        
    end
    return v
end

V (generic function with 1 method)

としておく。

行列を作成し、対角化すると、

In [4]:
Nx = 100
Ny = 100
L = 20.1
mat_H = make_matH(Nx,Ny,V,L)
@time e,v = eigs(mat_H,nev=20,which=:SM)
println(sort(real(e)))

  5.131422 seconds (6.84 M allocations: 424.800 MiB, 6.60% gc time)
[0.0569595, 0.144572, 0.144572, 0.259354, 0.259906, 0.299925, 0.400561, 0.400561, 0.484221, 0.484221, 0.566293, 0.566474, 0.695771, 0.697519, 0.736271, 0.75639, 0.75639, 0.936223, 0.936223, 0.97003]


と固有値が得られる。

QM04で解いた$n=0$の時の固有値は

In [5]:
println([0.057774, 0.304407, 0.748117, 1.389, 2.22705, 3.26228, 4.49467, 5.92421, 7.55088, 9.37468])

[0.057774, 0.304407, 0.748117, 1.389, 2.22705, 3.26228, 4.49467, 5.92421, 7.55088, 9.37468]


であり、$n=1$の固有値は

In [6]:
println([0.146673, 0.491691, 1.03395, 1.77341, 2.71005, 3.84386, 5.17484, 6.70296, 8.42821, 10.3506])

[0.146673, 0.491691, 1.03395, 1.77341, 2.71005, 3.84386, 5.17484, 6.70296, 8.42821, 10.3506]


である。二次元直交座標で解くことによって、異なる$n$の解がまとめて出てきていることがわかる。なお、$n=0$の最低固有値以外が縮退しているのは、$n$の正負で縮退があるからである。

なお、もう少し精度をあげると、

In [7]:
Nx = 200
Ny = 200
L = 20.1
mat_H = make_matH(Nx,Ny,V,L)
@time e,v = eigs(mat_H,nev=20,which=:SM)
println(sort(real(e)))

  3.451619 seconds (17.68 k allocations: 360.574 MiB, 4.70% gc time)
[0.0573951, 0.145703, 0.145703, 0.261462, 0.261977, 0.302363, 0.403898, 0.403898, 0.488333, 0.488333, 0.571103, 0.571474, 0.702129, 0.703579, 0.742888, 0.763228, 0.763228, 0.945023, 0.945023, 0.978886]


さらに値は近づく。